In [1]:
import os
import os.path as osp
import glob
import numpy as np
from PIL import Image
import onnxruntime as ort
from torchvision import transforms

## Predict With Onnx

In [2]:
IDX_TO_CLASS = {
    0: "Apple Scab Leaf",
    1: "Apple leaf",
    2: "Apple rust leaf",
    3: "Bell_pepper leaf",
    4: "Bell_pepper leaf spot",
    5: "Blueberry leaf",
    6: "Cherry leaf",
    7: "Corn Gray leaf spot",
    8: "Corn leaf blight",
    9: "Corn rust leaf",
    10: "Peach leaf",
    11: "Potato leaf early blight",
    12: "Potato leaf late blight",
    13: "Raspberry leaf",
    14: "Soyabean leaf",
    15: "Squash Powdery mildew leaf",
    16: "Strawberry leaf",
    17: "Tomato Early blight leaf",
    18: "Tomato Septoria leaf spot",
    19: "Tomato leaf",
    20: "Tomato leaf bacterial spot",
    21: "Tomato leaf late blight",
    22: "Tomato leaf mosaic virus",
    23: "Tomato leaf yellow virus",
    24: "Tomato mold leaf",
    25: "grape leaf",
    26: "grape leaf black rot",
}

In [3]:
input_folder = './input_images/'
input_image_paths = glob.glob(input_folder + '*.jpg') + glob.glob(input_folder + '*.jpeg')+ glob.glob(input_folder + '*.png')

In [4]:
data_transforms = [transforms.Compose([transforms.Resize((256, 256)),
                   transforms.ToTensor(),
                   transforms.Normalize(mean=[0.469, 0.536, 0.369],
                   std=[0.260, 0.244, 0.282])]),
                   
                   transforms.Compose([transforms.Resize((224, 224)),
                   transforms.ToTensor(),
                   transforms.Normalize(mean=[0.469, 0.536, 0.369],
                   std=[0.260, 0.244, 0.282])])]

ort_sess = ort.InferenceSession('ensemble.onnx')

for idx, img_path in enumerate(input_image_paths):
    image = Image.open(img_path).convert('RGB')

    im1 = data_transforms[0](image).unsqueeze(0)
    im2 = data_transforms[1](image).unsqueeze(0)

    output = ort_sess.run(None, {'input1': im1.numpy(), 'input2': im2.numpy()})

    y_pred = np.argmax(output)

    print(f'{idx+1})  IMG PATH = {img_path}\n    PRED = {IDX_TO_CLASS[y_pred]}\n')

1)  IMG PATH = ./input_images/apple_rust_leaf_1.jpg
    PRED = Apple rust leaf

2)  IMG PATH = ./input_images/soyabean_leaf_1.jpeg
    PRED = Soyabean leaf

3)  IMG PATH = ./input_images/apple_leaf_1.jpeg
    PRED = Cherry leaf

4)  IMG PATH = ./input_images/grape_leaf_1.jpeg
    PRED = grape leaf

5)  IMG PATH = ./input_images/potato_early_leaf_blight_1.jpeg
    PRED = Potato leaf early blight

